# How tall are they?

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

---

In [5]:
stature_df = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQXafjyIM7m42ktMDr4zoMTWM0NwYm-oDjdA5Aba0QQvG7BDaJnA2tuuqJiheUzOIo6qlJxu1A9aY5D/pub?gid=1344072865&single=true&output=csv"
)

In [6]:
stature_df.head()

,years,P5,P10,P25,P50,P75,P90,P95
0,2.00,79.259820,80.524765,82.635236,84.975555,87.311214,89.409512,90.663549
1,2.04,79.647769,80.919463,83.042132,85.397317,87.749175,89.863158,91.127070
2,2.13,80.442257,81.735410,83.894303,86.290263,88.683435,90.835048,92.121676
3,2.21,81.226662,82.536994,84.725920,87.157142,89.587509,91.774206,93.082541
4,2.29,81.995396,83.319685,85.533892,87.996018,90.460176,92.679688,94.008728


In [7]:
stature_df[["P5", "P10", "P25", "P50", "P75", "P90", "P95"]] = (
    stature_df[["P5", "P10", "P25", "P50", "P75", "P90", "P95"]] / 2.54
)

In [8]:
stature_df_melted = stature_df.melt(
    id_vars="years",
    value_vars=["P5", "P10", "P25", "P50", "P75", "P90", "P95"],
    value_name="height",
    var_name="percentile",
)

In [9]:
stature_df_melted.dtypes

years         float64
percentile     object
height        float64
dtype: object

In [10]:
girls_df = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQXafjyIM7m42ktMDr4zoMTWM0NwYm-oDjdA5Aba0QQvG7BDaJnA2tuuqJiheUzOIo6qlJxu1A9aY5D/pub?gid=862332793&single=true&output=csv"
)

In [12]:
girls_df.groupby(["kid"])["height"].max().reset_index(name="max_height")

,kid,max_height
0,Eva,60.00
1,Isa,49.25
2,Luna,44.25


In [13]:
lines = (
    alt.Chart(stature_df_melted)
    .mark_line(size=0.3, strokeDash=[2, 2])
    .encode(
        x=alt.X("years:Q", title="Age"),
        y=alt.Y(
            "height",
            axis=alt.Axis(tickCount=5),
            title="Inches",
            scale=alt.Scale(domain=[30, 70]),
        ),
        color=alt.Color(
            "percentile",
            legend=None,
            scale=alt.Scale(
                domain=["P5", "P10", "P25", "P50", "P75", "P90", "P95"],
                range=[
                    "black",
                    "#c9c9c9",
                    "#c9c9c9",
                    "black",
                    "#c9c9c9",
                    "#c9c9c9",
                    "black",
                ],
            ),
        ),
    )
)

circles = (
    alt.Chart(girls_df)
    .mark_circle()
    .encode(
        x="age",
        y=alt.Y("height", scale=alt.Scale(domain=[30, 70])),
        color=alt.Color(
            "kid",
            legend=alt.Legend(
                orient="none",
                legendX=-25,
                legendY=-80,
                direction="horizontal",
                titleAnchor="middle",
                title=" ",
            ),
        ),
    )
)

circles_paths = (
    alt.Chart(girls_df)
    .mark_line(size=0.4)
    .encode(
        x="age",
        y=alt.Y(
            "height",
            scale=alt.Scale(domain=[30, 70]),
        ),
        color=alt.Color("kid", legend=None),
    )
)

height = (
    alt.Chart(girls_df[girls_df["height"] == girls_df["height"].max()])
    .mark_circle()
    .encode(x="age", y=alt.Y("height", scale=alt.Scale(domain=[30, 70])))
)

text = height.mark_text(
    align="left",
    baseline="middle",
    dx=5,  # Nudges text to right so it doesn't appear on top of the bar
).encode(text="height:Q")


(lines + circles + circles_paths + text).resolve_scale(color="independent").properties(
    width=650, height=300, title="Hu-Stiles growth charts"
)

alt.LayerChart(...)